## Sales-data-analysis

ym's rendition of TalkPython's "Excel to Python Course: ch8-case-study"

### Objectives

This notebook is a continuation of "1-Data_Prep". Based on the data that we have tabulated in the previous notebook, we explore how we could structure the commission payout for the following year.

- We will consider applying different commission rates to the different sales channels, based on the effort required to manage these clients.
- We will take a closer look at the number of companies and sales volumes based on geographical distribution, and consider reorganising the sales agents' coverage areas.

### Data Sources
- 'final_data.xlsx' : Processed file from notebook 1-Data_Prep
- https://github.com/talkpython/excel-to-python-course/tree/master/code/ch8-case-study

### Changes
- 12-16-2021 : Started project

In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime

### File Locations

In [2]:
today = datetime.today()
src_file = Path.cwd() / "data" / "processed" / "customer_processed.xlsx"
report = Path.cwd() / "reports" / "report_v2.xlsx"

In [3]:
final_data = pd.read_excel(src_file, sheet_name = "final_data")
# total_sales_by_agent = pd.read_excel(src_file, sheet_name = "agent_sales")
# total_sales_by_agent_and_channel = pd.read_excel(src_file, sheet_name = "channel_sales")

### Recapping data tabulated in previous notebook

In [4]:
final_data

,sku,qty,invoice_total,company_name,channel,state,region,first_name,last_name,tenure,comm_rate,comm
0,SW200,4,60800.0,Bell Frontier Resource,retail,CA,West,Leonard,Malcolm,3.8,0.007906,480.66
1,PS501,4,108000.0,Speed Resource Vision,retail,MO,Midwest,Mona,Sutton,5.4,0.007906,853.80
2,SW500,1,12800.0,Venture Construction,retail,CT,NorthEast,Shannon,Muniz,5.6,0.007906,101.19
3,ACC5144,4,1408.0,Studio Pacific Galaxy,retail,TX,South,Mickey,Tyner,0.7,0.007906,11.13
4,SW200,1,16600.0,Vision People Solutions,retail,VA,South,Mickey,Tyner,0.7,0.007906,131.23
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,SPB1,1,4050.0,Software Bell Technology,retail,OH,Midwest,Mona,Sutton,5.4,0.007906,32.02
1996,SW500,3,45600.0,Resource Adventure Internet,retail,MI,Midwest,Mona,Sutton,5.4,0.007906,360.49
1997,SW200,2,24000.0,Vision People Solutions,retail,VA,South,Mickey,Tyner,0.7,0.007906,189.73
1998,ACC9011,18,4464.0,Advanced Alpha Federated,reseller,MN,Midwest,Mona,Sutton,5.4,0.007906,35.29


In [5]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sku            2000 non-null   object 
 1   qty            2000 non-null   int64  
 2   invoice_total  2000 non-null   float64
 3   company_name   2000 non-null   object 
 4   channel        2000 non-null   object 
 5   state          2000 non-null   object 
 6   region         2000 non-null   object 
 7   first_name     2000 non-null   object 
 8   last_name      2000 non-null   object 
 9   tenure         2000 non-null   float64
 10  comm_rate      2000 non-null   float64
 11  comm           2000 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 187.6+ KB


In [6]:
final_data.describe()

,qty,invoice_total,tenure,comm_rate,comm
count,2000.000000,2000.000000,2000.000000,2.000000e+03,2000.000000
mean,5.336000,63246.831000,3.077350,7.905534e-03,500.000025
std,6.072524,94703.387591,2.269759,1.596345e-16,748.680625
min,1.000000,240.000000,0.700000,7.905534e-03,1.900000
25%,2.000000,12400.000000,0.700000,7.905534e-03,98.030000
50%,3.000000,32000.000000,3.800000,7.905534e-03,252.980000
75%,4.000000,66450.000000,5.400000,7.905534e-03,525.325000
max,24.000000,705600.000000,5.600000,7.905534e-03,5578.150000


In [7]:
final_data.describe(include=object)

,sku,company_name,channel,state,region,first_name,last_name
count,2000,2000,2000,2000,2000,2000,2000
unique,12,50,3,31,4,4,4
top,SW200,Analysis Future Provider,retail,VA,South,Mickey,Tyner
freq,338,53,1503,146,931,931,931


### Perform Data Analysis

#### (1) Applying different commission rates to the different sales channels

As mentioned in the previous notebook, we will explore setting a different commission rate for each sales channel, based on the amount of effort required to maintain these clients.

It appears that the partner channel requires the least effort, followed by reseller, and lastly retail.

Let's try to work out the commission rates accordingly, while keeping within the $1mil budget.

In [8]:
sales_channels = final_data.groupby('channel').agg({'invoice_total':'sum'})
sales_channels.style.format('{:,.0f}')

,invoice_total
channel,
partner,"7,002,612"
reseller,"72,708,276"
retail,"46,782,774"


In [9]:
sales_channels['comm_rate'] = [0.005,0.0065,0.01]
sales_channels['total_comm'] = sales_channels['invoice_total'] * sales_channels['comm_rate']
sales_channels

,invoice_total,comm_rate,total_comm
channel,,,
partner,7002612.0,0.0050,35013.060
reseller,72708276.0,0.0065,472603.794
retail,46782774.0,0.0100,467827.740


In [10]:
sales_channels['total_comm'].sum()

975444.594

Using commission rates of 0.5%, 0.65% and 1% for partner, reseller and retail respectively gives us a total commission payout of \\$975,444.59, which is within our sales commission budget of \\$1mil. Let's apply this to the sales agents and observe how their commission distribution changes.

In [11]:
partner = (final_data['channel'] == 'partner')
reseller = (final_data['channel'] == 'reseller')
retail = (final_data['channel'] == 'retail')

In [12]:
final_data.loc[partner,'comm_rate'] = 0.005
final_data.loc[reseller,'comm_rate'] = 0.0065
final_data.loc[retail,'comm_rate'] = 0.01
final_data['comm'] = final_data['invoice_total'] * final_data['comm_rate']
final_data

,sku,qty,invoice_total,company_name,channel,state,region,first_name,last_name,tenure,comm_rate,comm
0,SW200,4,60800.0,Bell Frontier Resource,retail,CA,West,Leonard,Malcolm,3.8,0.0100,608.000
1,PS501,4,108000.0,Speed Resource Vision,retail,MO,Midwest,Mona,Sutton,5.4,0.0100,1080.000
2,SW500,1,12800.0,Venture Construction,retail,CT,NorthEast,Shannon,Muniz,5.6,0.0100,128.000
3,ACC5144,4,1408.0,Studio Pacific Galaxy,retail,TX,South,Mickey,Tyner,0.7,0.0100,14.080
4,SW200,1,16600.0,Vision People Solutions,retail,VA,South,Mickey,Tyner,0.7,0.0100,166.000
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,SPB1,1,4050.0,Software Bell Technology,retail,OH,Midwest,Mona,Sutton,5.4,0.0100,40.500
1996,SW500,3,45600.0,Resource Adventure Internet,retail,MI,Midwest,Mona,Sutton,5.4,0.0100,456.000
1997,SW200,2,24000.0,Vision People Solutions,retail,VA,South,Mickey,Tyner,0.7,0.0100,240.000
1998,ACC9011,18,4464.0,Advanced Alpha Federated,reseller,MN,Midwest,Mona,Sutton,5.4,0.0065,29.016


In [13]:
final_data['comm'].sum()

975444.594

In [14]:
final_data.groupby(['last_name']).agg({'comm':'sum'}).style.format('{:,.2f}')

,comm
last_name,
Malcolm,"101,610.90"
Muniz,"106,724.67"
Sutton,"401,671.53"
Tyner,"365,437.50"


The changes are minimal, but these commission rates makes more sense in business context.

#### (2) Geographical Distribution

Let's start by looking at how many states and companies each of the agents are currently covering, and the total sales.

In [15]:
final_data.groupby(['last_name','region']).agg({'state':'nunique','company_name':'nunique','invoice_total':'sum'}).style.format({'invoice_total':'{:,.0f}'})

,,state,company_name,invoice_total
last_name,region,,,
Malcolm,West,4,5,"12,909,772"
Muniz,NorthEast,5,6,"12,850,364"
Sutton,Midwest,8,15,"55,940,194"
Tyner,South,14,24,"44,793,332"


Sutton and Tyler are clearly better positioned to bring in a higher volume of sales, since they are both covering more than twice the number of states and more than 3 times the number of companies compared to Malcolm and Muniz. To address this misproportion, we shall try to allocate more states/companies to Malcolm and Muniz, to give them a fairer chance at generating commission earnings.

In [16]:
agent_vs_channel = pd.pivot_table(final_data, 
                                  index = ['last_name', 'region'],
                                  columns = 'channel',
                                  values = 'company_name',
                                  aggfunc = 'nunique',
                                  margins = True,
                                  fill_value = 0)
agent_vs_channel

,channel,partner,reseller,retail,All
last_name,region,,,,
Malcolm,West,0,1,4,5
Muniz,NorthEast,0,1,5,6
Sutton,Midwest,0,6,9,15
Tyner,South,2,2,20,24
All,,2,10,38,50


Based on the agent_vs_channel table, we have previously also discussed that the skew could be due to agent expertise or mere geographical preference. To identify the cause, we could reorganise the agents' area of coverage, by allocating some states that are currently weak in retail to Tyner and states that are currently weak in reseller to Tyner, and observe the changes in the following year's sales data.

Let's drill down further into how many companies in each sales channel there are in each state, and how much commission can be earned in each state.

In [17]:
state_vs_channel = pd.pivot_table(final_data, 
                                  index = ['last_name', 'region','state'],
                                  columns = ['channel'],
                                  values = ['company_name'],
                                  aggfunc = 'nunique',
                                  margins = True,
                                  fill_value = 0)
state_vs_channel

company_name                    
channel                        partner reseller retail All
last_name region    state                                 
Malcolm   West      AZ               0        0      1   1
                    CA               0        0      2   2
                    ID               0        1      0   1
                    OR               0        0      1   1
Muniz     NorthEast CT               0        0      2   2
                    MA               0        0      1   1
                    ME               0        0      1   1
                    NY               0        0      1   1
                    PA               0        1      0   1
Sutton    Midwest   IA               0        1      0   1
                    IL               0        1      0   1
                    KS               0        1      2   3
                    MI               0        0      3   3
                    MN               0        1      0   1
                    MO               0        1      2   3
                    NE               0        1      1   2
                    OH               0        0      1   1
Tyner     South     AL               0        0      1   1
                    DC               1        0      1   2
                    FL               1        0      1   2
                    GA               0        0      1   1
                    KY               0        0      1   1
                    LA               0        1      0   1
                    MD               0        0      1   1
                    MS               0        0      1   1
                    NC               0        1      1   2
                    OK               0        0      1   1
                    SC               0        0      2   2
                    TX               0        0      2   2
                    VA               0        0      4   4
                    WV               0        0      3   3
All                                  2       10     38  50

In [18]:
final_data.groupby(['last_name','region','state']).agg({'company_name':'nunique','comm':'sum'}).sort_values(by=['last_name','comm'],ascending=[True,False]).style.format('{:,.0f}')

Taking into account geographical proximity for the agents' ease of management, to address Malcolm and Muniz's lack of access to companies, we could try allocating NE, TX and OK to Malcolm, and MI, OH and VA to Muniz.

To swap some of the reseller-heavy and retail-heavy states, we could reallocate IL to Tyner and WV and KY to Sutton.

Let's take a look at how this will turn out.

In [19]:
final_data_v2 = final_data.iloc[:,[0,1,2,3,4,5,6,8,10,11]].copy()
ne = (final_data_v2['state'] == 'NE')
tx = (final_data_v2['state'] == 'TX')
mi = (final_data_v2['state'] == 'MI')
oh = (final_data_v2['state'] == 'OH')
va = (final_data_v2['state'] == 'VA')
ok = (final_data_v2['state'] == 'OK')

il = (final_data_v2['state'] == 'IL')
wv = (final_data_v2['state'] == 'WV')
ky = (final_data_v2['state'] == 'KY')

final_data_v2.loc[ne,'last_name'] = 'Malcolm'
final_data_v2.loc[tx,'last_name'] = 'Malcolm'
final_data_v2.loc[ok,'last_name'] = 'Malcolm'
final_data_v2.loc[mi,'last_name'] = 'Muniz'
final_data_v2.loc[oh,'last_name'] = 'Muniz'
final_data_v2.loc[va,'last_name'] = 'Muniz'

final_data_v2.loc[il,'last_name'] = 'Tyner'
final_data_v2.loc[wv,'last_name'] = 'Sutton'
final_data_v2.loc[ky,'last_name'] = 'Sutton'

final_data_v2

,sku,qty,invoice_total,company_name,channel,state,region,last_name,comm_rate,comm
0,SW200,4,60800.0,Bell Frontier Resource,retail,CA,West,Malcolm,0.0100,608.000
1,PS501,4,108000.0,Speed Resource Vision,retail,MO,Midwest,Sutton,0.0100,1080.000
2,SW500,1,12800.0,Venture Construction,retail,CT,NorthEast,Muniz,0.0100,128.000
3,ACC5144,4,1408.0,Studio Pacific Galaxy,retail,TX,South,Malcolm,0.0100,14.080
4,SW200,1,16600.0,Vision People Solutions,retail,VA,South,Muniz,0.0100,166.000
...,...,...,...,...,...,...,...,...,...,...
1995,SPB1,1,4050.0,Software Bell Technology,retail,OH,Midwest,Muniz,0.0100,40.500
1996,SW500,3,45600.0,Resource Adventure Internet,retail,MI,Midwest,Muniz,0.0100,456.000
1997,SW200,2,24000.0,Vision People Solutions,retail,VA,South,Muniz,0.0100,240.000
1998,ACC9011,18,4464.0,Advanced Alpha Federated,reseller,MN,Midwest,Sutton,0.0065,29.016


In [20]:
total_sales_by_agent_and_ch_v2 = pd.pivot_table(final_data_v2, 
                                                  index = 'last_name', 
                                                  columns = 'channel',
                                                  values = 'invoice_total',
                                                  aggfunc = 'sum',
                                                  margins = True,
                                                  fill_value = 0).style.format('{:,.0f}')
total_sales_by_agent_and_ch_v2

channel,partner,reseller,retail,All
last_name,,,,
Malcolm,0,"16,974,972","10,370,388","27,345,360"
Muniz,0,"6,222,564","17,167,132","23,389,696"
Sutton,0,"28,985,736","9,074,962","38,060,698"
Tyner,"7,002,612","20,525,004","10,170,292","37,697,908"
All,"7,002,612","72,708,276","46,782,774","126,493,662"


After the proposed changes, all 4 agents have a more balanced portfolio in terms of sales volume, with Sutton and Tyner still maintaning the lead.

In [21]:
final_data_v2.groupby(['last_name']).agg({'state':'nunique','company_name':'nunique','invoice_total':'sum'}).style.format({'invoice_total':'{:,.0f}'})

,state,company_name,invoice_total
last_name,,,
Malcolm,7,10,"27,345,360"
Muniz,8,14,"23,389,696"
Sutton,6,12,"38,060,698"
Tyner,10,14,"37,697,908"


In [22]:
agent_vs_channel_v2 = pd.pivot_table(final_data_v2, 
                                  index = ['last_name'],
                                  columns = 'channel',
                                  values = 'company_name',
                                  aggfunc = 'nunique',
                                  margins = True,
                                  fill_value = 0)
agent_vs_channel_v2

channel,partner,reseller,retail,All
last_name,,,,
Malcolm,0,2,8,10
Muniz,0,1,13,14
Sutton,0,4,8,12
Tyner,2,3,9,14
All,2,10,38,50


The number of states and companies that all 4 agents have access to is also more balanced. Sutton and Tyner now have about the same number of reseller and retail clients on their portfolios. If these proposed changes were to be applied, we can observe to see if the number of resellers and retailers grows in the next year.

In [23]:
agent_comm_v2 = final_data_v2.groupby(['last_name']).agg({'comm':'sum'}).style.format('{:,.2f}')
agent_comm_v2

,comm
last_name,
Malcolm,"214,041.20"
Muniz,"212,117.99"
Sutton,"279,156.90"
Tyner,"270,128.51"


The commission earnings between all 4 agents have become more evenly distributed, while still allowing Sutton and Tyner to maintain their lead.

### Discussion points

- Without any historical data of the 4 agents' prior track record, we are unable to determine if the disparity in the number of states and companies covered by the 4 agents is due to their past performance, or a legacy issue.
- We also have no data on the sales agents' relationship with their customers. There is a risk of a negative impact towards business with some customers if we try to change their sales agents.
- These proposed changes allows all 4 agents to start off the upcoming year on a more leveled playing field.
- The performance of all 4 agents should be monitored and compared against their previous year's performance.
- In particular, Malcolm and Muniz's performance should be compared against each other's, while Sutton's should be compared against Tyner.

### Conclusion

- Commission rate of 0.5%, 0.65% and 1% to be applied towards the partner, reseller and retail channels respectively.
- Reallocate the states NE, TX and OK to Malcolm, and MI, OH and VA to Muniz to address their existing lack of customers on their portfolios.
- Reallocate IL to Tyner, and WV and KY to Sutton so that Tyner could grow the retail business in IL while Sutton could grow the reseller business in WV and KY.


### Save Excel file into reports directory

These tables will be saved into the report_v2 file, for ease of comparison against report_v1.

In [24]:
writer = pd.ExcelWriter(report, engine='xlsxwriter')
final_data_v2.to_excel(writer, index = False, sheet_name="final_data_v2")
agent_vs_channel_v2.to_excel(writer, index = True, sheet_name="agents_channels")
total_sales_by_agent_and_ch_v2.to_excel(writer, index = True, sheet_name="agents_channels", startcol = 9)
agent_comm_v2.to_excel(writer, index = True, sheet_name="agents_comms")

workbook = writer.book

num_format = workbook.add_format({'num_format': '#,##0'})
worksheet = writer.sheets["agents_channels"]
worksheet.set_column('K:N',11, num_format)

num_format = workbook.add_format({'num_format': '#,##0.00'})
worksheet = writer.sheets["agents_comms"]
worksheet.set_column('B:B',10, num_format)
worksheet = writer.sheets['final_data_v2']
worksheet.set_column('C:C',14, num_format)
worksheet.set_column('D:D',28)

writer.close()